In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv(r'E:\Santander Recommendation\train_ver2.csv')
test = pd.read_csv(r'E:\Santander Recommendation\test_ver2.csv')

C:\Users\liamc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\liamc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.fecha_dato = pd.to_datetime(train.fecha_dato, format="%Y-%m-%d")
train.fecha_alta = pd.to_datetime(train.fecha_alta, format="%Y-%m-%d")
test.fecha_dato = pd.to_datetime(test.fecha_dato, format="%Y-%m-%d")
test.fecha_alta = pd.to_datetime(test.fecha_alta, format="%Y-%m-%d")

In [4]:
months = train.fecha_dato.unique()

In [5]:
train_wnewproducts = pd.DataFrame()

In [6]:
i = 1
while i < len(months):
    #select sequential months
    new_month = train[train.fecha_dato == months[i]]
    prev_month = train[train.fecha_dato == months[i-1]]
    #select only customers present in both month
    new_month = new_month.loc[new_month['ncodpers'].isin(prev_month.ncodpers)]
    prev_month = prev_month.loc[prev_month['ncodpers'].isin(new_month.ncodpers)]
    #sort by customer id so both months will be ordered the same
    new_month.sort_values(by = 'ncodpers', inplace = True)
    prev_month.sort_values(by = 'ncodpers', inplace = True)
    #pick out product columns
    product_cols = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1',
                'ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1',
                'ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_dela_fin_ult1','ind_deme_fin_ult1',
                'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_nom_pens_ult1',
                'ind_nomina_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1',
                'ind_recibo_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1']
    #new dataframe with just product
    new_month_products = new_month[product_cols]
    #add in the corresponding customer ids
    new_month_products['ncodpers'] = new_month.ncodpers
    #get rid of index
    new_month_products.reset_index(drop=True,inplace=True)
    #now repeat for the previous month but make ids zero so a subtraction works 
    prev_month_products = prev_month[product_cols]
    prev_month_products['ncodpers'] = prev_month.ncodpers
    prev_month_products.ncodpers = 0
    prev_month_products.reset_index(drop=True,inplace=True)
    #subtract to find the new products
    new_products = new_month_products.subtract(prev_month_products)
    #remove products that the customer dropped fill NA's
    new_products[new_products < 0] = 0
    new_products = new_products.fillna(0)
    #merge the new products features with the month data
    new_products = new_products.merge(new_month.iloc[:,0:24], on='ncodpers')
    #add the interated months into training dataframe
    train_wnewproducts = pd.concat([train_wnewproducts,new_products], axis = 0)
    print("progress = ", float(i/len(months)))
    i += 1

C:\Users\liamc\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\liamc\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\liamc\Anaconda3\lib\site-packages\pandas\core\generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

progress =  0.058823529411764705
progress =  0.11764705882352941
progress =  0.17647058823529413
progress =  0.23529411764705882
progress =  0.29411764705882354
progress =  0.35294117647058826
progress =  0.4117647058823529
progress =  0.47058823529411764
progress =  0.5294117647058824
progress =  0.5882352941176471
progress =  0.6470588235294118
progress =  0.7058823529411765
progress =  0.7647058823529411
progress =  0.8235294117647058
progress =  0.8823529411764706
progress =  0.9411764705882353


In [7]:
train_wnewproducts = train_wnewproducts.loc[(train_wnewproducts.iloc[:,0:24] != 0).any(axis=1)]

In [8]:
final_month = train[train.fecha_dato == '2016-05-28']

In [9]:
df = pd.concat([train_wnewproducts,test], axis = 0, ignore_index = True)
train = []

In [10]:
dirtyrows = df[df.ind_empleado.isnull()].index

df = df.drop(dirtyrows, axis = 0)

df.canal_entrada = df.canal_entrada.fillna("Unknown")

df = df.drop("cod_prov", 1)

df.conyuemp = df.conyuemp.fillna("Unknown")

df.loc[df.indrel_1mes == '1', 'indrel_1mes'] = 'Primary'
df.loc[df.indrel_1mes == '1.0', 'indrel_1mes'] = 'Primary'
df.loc[df.indrel_1mes == 1, 'indrel_1mes'] = 'Primary'
df.loc[df.indrel_1mes == 1.0, 'indrel_1mes'] = 'Primary'

df.loc[df.indrel_1mes == '2', 'indrel_1mes'] = 'Co-owner'
df.loc[df.indrel_1mes == '2.0', 'indrel_1mes'] = 'Co-owner'
df.loc[df.indrel_1mes == 2, 'indrel_1mes'] = 'Co-owner'
df.loc[df.indrel_1mes == 2.0, 'indrel_1mes'] = 'Co-owner'

df.loc[df.indrel_1mes == '3', 'indrel_1mes'] = 'Former Primary'
df.loc[df.indrel_1mes == '3.0', 'indrel_1mes'] = 'Former Primary'
df.loc[df.indrel_1mes == 3, 'indrel_1mes'] = 'Former Primary'
df.loc[df.indrel_1mes == 3.0, 'indrel_1mes'] = 'Former Primary'

df.loc[df.indrel_1mes == '4', 'indrel_1mes'] = 'Former Co-owner'
df.loc[df.indrel_1mes == '4.0', 'indrel_1mes'] = 'Former Co-owner'
df.loc[df.indrel_1mes == 4, 'indrel_1mes'] = 'Former Co-owner'
df.loc[df.indrel_1mes == 4.0, 'indrel_1mes'] = 'Former Co-owner'

df.indrel_1mes  = df.indrel_1mes.fillna('Primary')

df.nomprov = df.nomprov.fillna("MADRID")

df.loc[df.renta == '         NA',"renta"] = 0
df.renta = df.renta.astype(float)
df.loc[df.renta == 0, 'renta'] = df[df.renta > 0].groupby('nomprov').renta.transform('median')
df.loc[df.renta.isnull(), "renta"] = df.groupby('nomprov').renta.transform('median')

df.segmento = df[df.renta <= 98000].segmento.fillna("03 - UNIVERSITARIO")
df.segmento = df[df.renta <= 125500].segmento.fillna("02 - PARTICULARES")
df.segmento = df.segmento.fillna("01 - TOP")

df.sexo = df.sexo.fillna("V")

df.tiprel_1mes = df.tiprel_1mes.fillna('I')

df.ind_nomina_ult1 = df.ind_nomina_ult1.fillna(0.0)
df.ind_nom_pens_ult1 = df.ind_nom_pens_ult1.fillna(0.0)

In [11]:
df.loc[df.antiguedad == -999999, 'antiguedad'] = df[df.antiguedad.astype(float) >= 0].antiguedad.median()

In [12]:
df['ageGroup'] = (df.age.astype(float) // 10) - 1

df['isSpanish'] = df.pais_residencia.map(lambda x: 1 if x == "ES" else 0)

df['majorCity'] = df.nomprov.map(lambda x: 1 if x == "MADRID" or x == "BARCELONA" else 0)

df['fecha_alta_year'] = pd.DatetimeIndex(df.fecha_alta).year - 1995
df['fecha_dato_year'] = pd.DatetimeIndex(df.fecha_dato).year - 2015
df['fecha_alta_month'] = pd.DatetimeIndex(df.fecha_alta).month - 1
df['fecha_dato_month'] = pd.DatetimeIndex(df.fecha_dato).month - 1

df.antiguedad = df.antiguedad.astype(int)
df['antiguedad_years'] = df.antiguedad // 12

df.loc[df.indrel == 99.0, "indrel"] = 0.0

In [13]:
df.ult_fec_cli_1t.fillna(0)
df['HAS_ult_fec_cli_1t'] = df.ult_fec_cli_1t.map(lambda x: 1 if type(x)==str else 0)

df = df.drop('ult_fec_cli_1t', 1)

df['rentaGroup'] = df.renta.astype(float) // 50000
df.loc[df.renta >= 1000000, "rentaGroup"] = 20
df.loc[df.renta >= 10000000, "rentaGroup"] = 21

In [14]:
l = len(train_wnewproducts)- len(dirtyrows)

In [15]:
train_final, test = df[:l], df[l:]
train_final_training_cols = train_final
train_final_training_cols = train_final_training_cols.drop(product_cols, axis=1)
test = test.drop(product_cols, axis=1)

In [16]:
print(len(train_final))
print(len(test))

446022
929615


In [17]:
train_final.ncodpers.isin(test.ncodpers).sum()

441020

In [18]:
df = pd.concat([train_final_training_cols, test], axis = 0)

In [19]:
df.pais_residencia = df.pais_residencia.astype('category').cat.codes
df.canal_entrada = df.canal_entrada.astype('category').cat.codes
df.nomprov = df.nomprov.astype('category').cat.codes
final_month.nomprov = final_month.indrel_1mes.astype('category').cat.codes
df = pd.get_dummies(df, columns = ['ind_empleado','sexo','tiprel_1mes','indresi',
                                   'indext','conyuemp','indfall','segmento','indrel_1mes'])

df = df.drop(['fecha_dato', 'fecha_alta'], axis = 1)

In [20]:
train_final_training_cols, test = df[:l], df[l:]
df=[]

In [21]:
labels = train_final[product_cols]
labels['ncodpers'] = train_final.ncodpers
labels = labels.set_index("ncodpers")
stacked_labels = labels.stack()
filtered_labels = stacked_labels.reset_index()
filtered_labels.columns = ["ncodpers", "product", "newly_added"]
filtered_labels = filtered_labels[filtered_labels["newly_added"] == 1]
multiclass_train = filtered_labels.merge(train_final_training_cols, on="ncodpers", how="left")
train_final = multiclass_train.drop_duplicates(multiclass_train, keep='last')
labels_final = train_final['product']
train_final_ncodpers = train_final.ncodpers

C:\Users\liamc\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
#insert some collaborative filtering features part 1
train = pd.read_csv(r'E:\Santander Recommendation\train_ver2.csv')
lastmonth = train.loc[train['fecha_dato']=='2016-05-28']
train=[]
lastmonthlabels = lastmonth[product_cols]
lastmonthlabels['ncodpers'] = lastmonth.ncodpers #check if another last month
lastmonthlabels = lastmonthlabels.set_index('ncodpers')
stacked_lastmonthlabels = lastmonthlabels.stack()
filtered_lastmonthlabels = stacked_lastmonthlabels.reset_index()
filtered_lastmonthlabels.columns = ["ncodpers", "product", "is_owned"]
lastmonthlabels=[]

C:\Users\liamc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\liamc\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [23]:
#CF part 2
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import train_test_split
reader = Reader(line_format='user item rating', rating_scale=(0,1))
data = Dataset.load_from_df(filtered_lastmonthlabels, reader=reader)
trainset, testset = train_test_split(data, test_size=.05)
algo = SVD(n_factors=50,verbose=True)
algo.fit(trainset)
predictions = algo.test(testset)
print(accuracy.rmse(predictions))
latentfeatures = pd.DataFrame(algo.pu)
latentfeatures['ncodpers'] = lastmonth.ncodpers.values

RMSE: 0.1917
0.19168434530058975


In [24]:
#CF part 3: add the features
test = pd.merge(test, latentfeatures,how='inner',on='ncodpers')

In [25]:
train_final = pd.merge(train_final, latentfeatures, how='inner', on='ncodpers')

In [26]:
labels_final = train_final['product']
train_final_ncodpers = train_final.ncodpers
train_final = train_final.drop(['ncodpers','newly_added','product'], axis = 1)

In [27]:
final_month = pd.merge(final_month,latentfeatures)

In [28]:
#need to go through and describe each step
dirtyrows = final_month[final_month.ind_empleado.isnull()].index
final_month = final_month.drop(dirtyrows, axis = 0)
final_month.canal_entrada = final_month.canal_entrada.fillna("Unknown")
final_month = final_month.drop("cod_prov", 1)
final_month.conyuemp = final_month.conyuemp.fillna("Unknown")
final_month.loc[final_month.indrel_1mes == '1', 'indrel_1mes'] = 'Primary'
final_month.loc[final_month.indrel_1mes == '1.0', 'indrel_1mes'] = 'Primary'
final_month.loc[final_month.indrel_1mes == 1, 'indrel_1mes'] = 'Primary'
final_month.loc[final_month.indrel_1mes == 1.0, 'indrel_1mes'] = 'Primary'
final_month.loc[final_month.indrel_1mes == '2', 'indrel_1mes'] = 'Co-owner'
final_month.loc[final_month.indrel_1mes == '2.0', 'indrel_1mes'] = 'Co-owner'
final_month.loc[final_month.indrel_1mes == 2, 'indrel_1mes'] = 'Co-owner'
final_month.loc[final_month.indrel_1mes == 2.0, 'indrel_1mes'] = 'Co-owner'
final_month.loc[final_month.indrel_1mes == '3', 'indrel_1mes'] = 'Former Primary'
final_month.loc[final_month.indrel_1mes == '3.0', 'indrel_1mes'] = 'Former Primary'
final_month.loc[final_month.indrel_1mes == 3, 'indrel_1mes'] = 'Former Primary'
final_month.loc[final_month.indrel_1mes == 3.0, 'indrel_1mes'] = 'Former Primary'
final_month.loc[final_month.indrel_1mes == '4', 'indrel_1mes'] = 'Former Co-owner'
final_month.loc[final_month.indrel_1mes == '4.0', 'indrel_1mes'] = 'Former Co-owner'
final_month.loc[final_month.indrel_1mes == 4, 'indrel_1mes'] = 'Former Co-owner'
final_month.loc[final_month.indrel_1mes == 4.0, 'indrel_1mes'] = 'Former Co-owner'
final_month.indrel_1mes = final_month.indrel_1mes.fillna('Primary')
final_month.nomprov = final_month.nomprov.fillna("MADRID")
final_month.renta = final_month.renta.astype(float)
final_month.loc[final_month.renta.isnull(), "renta"] = final_month.groupby('nomprov').renta.transform('median')
final_month.segmento = final_month[final_month.renta <= 98000].segmento.fillna("03 - UNIVERSITARIO")
final_month.segmento = final_month[final_month.renta <= 125500].segmento.fillna("02 - PARTICULARES")
final_month.segmento = final_month.segmento.fillna("01 - TOP")
final_month.sexo = final_month.sexo.fillna("V")
final_month.tiprel_1mes = final_month.tiprel_1mes.fillna('I')
final_month.ind_nomina_ult1 = final_month.ind_nomina_ult1.fillna(0.0)
final_month.ind_nom_pens_ult1 = final_month.ind_nom_pens_ult1.fillna(0.0)
final_month.loc[final_month.antiguedad == -999999, 'antiguedad'] = final_month[final_month.antiguedad >= 0].antiguedad.median()
final_month.age = final_month.age.astype(int)
final_month.loc[final_month.age < 20,"age"] = 19
final_month.loc[final_month.age > 90,"age"] = 91
final_month['ageGroup'] = (final_month.age // 10) - 1
final_month['isSpanish'] = final_month.pais_residencia.map(lambda x: 1 if x == "ES" else 0)
final_month['majorCity'] = final_month.nomprov.map(lambda x: 1 if x == "MADRID" or x == "BARCELONA" else 0)
final_month['fecha_alta_year'] = pd.DatetimeIndex(final_month.fecha_alta).year - 1995
final_month['fecha_dato_year'] = pd.DatetimeIndex(final_month.fecha_dato).year - 2015
final_month['fecha_alta_month'] = pd.DatetimeIndex(final_month.fecha_alta).month - 1
final_month['fecha_dato_month'] = pd.DatetimeIndex(final_month.fecha_dato).month - 1
final_month.antiguedad = final_month.antiguedad.astype(int)
final_month['antiguedad_years'] = final_month.antiguedad // 12
final_month.loc[final_month.indrel == 99.0, "indrel"] = 0.0
final_month.ult_fec_cli_1t.fillna(0)
final_month['HAS_ult_fec_cli_1t'] = final_month.ult_fec_cli_1t.map(lambda x: 1 if type(x) == str else 0)
final_month = final_month.drop('ult_fec_cli_1t', 1)
final_month['rentaGroup'] = final_month.renta.astype(float) // 50000
final_month.loc[final_month.renta >= 1000000, "rentaGroup"] = 20
final_month.loc[final_month.renta >= 10000000, "rentaGroup"] = 21
final_month_training_cols = final_month
final_month_training_cols = final_month_training_cols.drop(product_cols, axis=1)
final_month.pais_residencia = final_month.pais_residencia.astype('category').cat.codes
final_month.canal_entrada = final_month.canal_entrada.astype('category').cat.codes
final_month.nomprov = final_month.nomprov.astype('category').cat.codes
final_month.nomprov = final_month.indrel_1mes.astype('category').cat.codes
final_month = pd.get_dummies(final_month, columns = ['ind_empleado','sexo','tiprel_1mes','indresi',
                                   'indext','conyuemp','indfall','segmento','indrel_1mes'])
final_month = final_month.drop(['fecha_dato', 'fecha_alta'], axis = 1)

In [29]:
labels_final_month = final_month[product_cols]
labels_final_month['ncodpers'] = final_month.ncodpers
labels_final_month = labels_final_month.set_index("ncodpers")
stacked_labels_final_month = labels_final_month.stack()
filtered_labels_final_month = stacked_labels_final_month.reset_index()
filtered_labels_final_month.columns = ["ncodpers", "product", "newly_added"]
filtered_labels_final_month = filtered_labels_final_month[filtered_labels_final_month["newly_added"] == 1]
multiclass_final_month = filtered_labels_final_month.merge(final_month_training_cols, on="ncodpers", how="left")
final_month = multiclass_final_month.drop_duplicates(multiclass_final_month, keep='last')
labels_final_month_final = final_month['product']
final_month_ncodpers = final_month.ncodpers
final_month = final_month.drop(['ncodpers','newly_added','product'], axis = 1)

C:\Users\liamc\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [30]:
print(len(train_final))
print(len(labels_final))
print(len(final_month))
print(len(labels_final_month_final))

1013148
1013148
1240538
1240538


In [31]:
labels_final.value_counts()

ind_recibo_ult1      223500
ind_nom_pens_ult1    151815
ind_nomina_ult1      135539
ind_cco_fin_ult1     121625
ind_tjcr_fin_ult1    115884
ind_cno_fin_ult1      93696
ind_ecue_fin_ult1     58871
ind_dela_fin_ult1     26004
ind_reca_fin_ult1     24385
ind_ctma_fin_ult1     15534
ind_valo_fin_ult1     12290
ind_ctop_fin_ult1      8839
ind_fond_fin_ult1      8748
ind_ctpp_fin_ult1      6438
ind_deco_fin_ult1      5935
ind_plan_fin_ult1      1791
ind_deme_fin_ult1       600
ind_ctju_fin_ult1       505
ind_pres_fin_ult1       406
ind_cder_fin_ult1       344
ind_hip_fin_ult1        199
ind_viv_fin_ult1        191
ind_aval_fin_ult1         6
ind_ahor_fin_ult1         3
Name: product, dtype: int64

In [32]:
labels_final = labels_final.astype('category').cat.codes

In [33]:
test = test.drop('ncodpers', axis = 1)

In [34]:
train_wnewproducts = []
data = []
trainset = []
testset = []
algo = []
predictions = []
latentfeatures = []

In [203]:
from sklearn.multioutput import MultiOutputClassifier
from keras.utils import to_categorical
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

In [36]:
y = to_categorical(labels_final)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(train_final, y, test_size=0.2, random_state=21)

In [39]:
X_train.shape

(810518, 95)

In [204]:
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X_train, y_train)
model = SelectFromModel(lsvc, prefit=True)

ValueError: bad input shape (810518, 24)

In [ ]:
model = SelectFromModel(clf, prefit=True)
X_train = model.transform(X_train)

In [ ]:
X_train.shape

In [172]:
forest = RandomForestClassifier(n_estimators=50,oob_score=True,max_features=.33,random_state=1,verbose=2)

In [173]:
multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)

In [174]:
multi_target_forest.fit(X_train,y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.33, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=True, random_state=1, verbose=0, warm_start=False),
           n_jobs=-1)

In [175]:
from sklearn.metrics import log_loss

In [176]:
y_pred_prob = multi_target_forest.predict_proba(X_test)

In [177]:
y_pred_prob[0]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [178]:
compiledclassprobs = pd.DataFrame(y_pred_prob[0][:,1])

In [179]:
for i in range(1,24):
    s = pd.Series(y_pred_prob[i][:,1])
    compiledclassprobs = pd.concat([compiledclassprobs,s],axis=1)

In [180]:
log_loss(y_test,compiledclassprobs.values)

7.841843796345737

In [181]:
import matplotlib.pyplot as plt

In [182]:
labels_final_month_final_cat = labels_final_month_final.astype('category').cat.codes

In [183]:
used_products = pd.DataFrame()
used_products['product'] = labels_final_month_final_cat
used_products['ncodpers'] = final_month_ncodpers
used_products = used_products.drop_duplicates(keep = 'last')

In [184]:
#create a dictionary to store each product a customer already has
used_recommendation_products = {}
product_cols = np.array(product_cols)
#iterate through used_products and add each one to used_recommendation_products
for idx,row_val in used_products.iterrows():
    used_recommendation_products.setdefault(row_val['ncodpers'],[]).append(product_cols[row_val['product']])

In [185]:
used_recommendation_products[15889]

['ind_cco_fin_ult1',
 'ind_ctpp_fin_ult1',
 'ind_tjcr_fin_ult1',
 'ind_valo_fin_ult1']

In [186]:
modeltest = normalize(test)

In [187]:
modelpredsraw = multi_target_forest.predict_proba(modeltest)

In [188]:
compiledclassprobs = pd.DataFrame(modelpredsraw[0][:,1])

In [189]:
for i in range(1,24):
    s = pd.Series(modelpredsraw[i][:,1])
    compiledclassprobs = pd.concat([compiledclassprobs,s],axis=1)

In [190]:
modelpreds = compiledclassprobs.values

In [191]:
pred = np.argsort(modelpreds, axis=1)
pred = np.fliplr(pred)

In [192]:
pred[0]

array([21, 20, 13,  2, 22, 10, 15, 12,  6, 16, 23,  7,  1,  3,  4,  5, 11,
        8,  9, 14, 17, 18, 19,  0], dtype=int64)

In [193]:
test_ids = np.array(pd.read_csv(r'E:\Santander Recommendation\test_ver2.csv',usecols=['ncodpers'])['ncodpers'])
product_cols = np.array(product_cols)
final_preds = []
#iteratively create the final rank ordered recommendations for each user, exluding ones the customer already has
for idx,predicted in enumerate(pred):
    ids = test_ids[idx]
    top_product = product_cols[predicted]
    used_products = used_recommendation_products.get(ids,[])
    new_top_product = []
    for product in top_product:
        if product not in used_products:
            new_top_product.append(product)
        if len(new_top_product) == 7:
            break
    final_preds.append(' '.join(new_top_product))

In [194]:
submission = pd.DataFrame({'ncodpers':test_ids,'added_products':final_preds})
submission.to_csv('submission.csv',index=False)

In [195]:
submission.shape

(929615, 2)

In [200]:
submission=[]
final_preds=[]
compiledclassprobs=[]